In [1]:
# https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
# https://www.tensorflow.org/guide/keras/working_with_rnns

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import load_model
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D
import joblib

In [2]:
dataset = pd.read_csv('../../data/preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
dataset = dataset.sample(frac=0.1, random_state=42)
dataset

,Text,Source,Human
287452,ING AsiaPacific Companys Problems Research Pap...,Human,1
222199,Crisis Love Inquiry Essay Critical Writing fol...,Human,1
453600,Sure sex segregation makes lot sense many spor...,Human,1
276338,Christianity Islam Values Essay Christianity f...,Human,1
78217,Becca liked swim practiced everyday hours ente...,GLM-130B,0
...,...,...,...
663613,Mass Eoghan Chada 10 brother Ruairi 5 said St ...,OPT-30B,0
285976,Asian Teachers Polish Lesson Perfection Stigle...,Human,1
679335,Move knife slowly avoid slipping accidentally ...,OPT-6.7B,0
775773,Good dreams likely occur person feeling relaxe...,Text-Davinci-003,0


In [3]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Text'])

In [4]:
# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(dataset['Text'])

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [5]:
# unitializing label encoder
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Human'])

# labels to numerical format conversion
encoded_labels = label_encoder.transform(dataset['Human'])

In [16]:
# Convolutional instantiation 1 Conv Layer
conv = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    Conv1D(128, 1, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

In [17]:
print(np.unique(encoded_labels))

[0 1]


In [18]:
conv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# training phase
conv.fit(padded_sequences, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7519 - loss: 0.4838 - val_accuracy: 0.8069 - val_loss: 0.3874
Epoch 2/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8352 - loss: 0.3495 - val_accuracy: 0.8185 - val_loss: 0.3719
Epoch 3/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8620 - loss: 0.3029 - val_accuracy: 0.8234 - val_loss: 0.3731
Epoch 4/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8871 - loss: 0.2571 - val_accuracy: 0.8270 - val_loss: 0.3770
Epoch 5/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9117 - loss: 0.2145 - val_accuracy: 0.8249 - val_loss: 0.3983
Epoch 6/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9315 - loss: 0.1786 - val_accuracy: 0.8233 - val_loss: 0.4246
Epoch 7/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9475 - loss: 0.1436 - val_accuracy: 0.8195 - val_loss: 0.4577
Epoch 8/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9612 - loss: 0.1124 

In [ ]:
# 0.9843

In [ ]:
conv.save('../../models/NeuralNetworks/conv_one_98.keras')